In [1]:
# importer les librairies nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import glob
import missingno as msno
import seaborn as sns
import numpy as np

In [2]:
#path = r'../input/laptop-price/laptop_price.csv'
path1 = r'../input/laptop-price-prediction-dataset/laptop_data.csv'
Data = pd.read_csv(path1, index_col=None, header=0, encoding='latin-1')
#Data = pd.concat(li, axis=0, ignore_index=True, sort=True)
copy_data1=Data
print(Data.shape)
Data.head()

# variables qualitatives

In [3]:
for col in Data.select_dtypes('object'):
    print(f'{col:-<50}{Data[col].unique()}')

In [4]:
print(Data.dtypes)
print(Data.dtypes.value_counts())
print(Data.shape)

In [5]:
plt.figure(figsize=(20,10))
sns.heatmap(Data.isna(),cbar=False)

In [6]:
def count_percent(data):
  df_cols = pd.DataFrame({'Count Missing': data.isnull().sum(),
                        'Percent Missing': data.isnull().sum()*100/data.shape[0]})
  return df_cols

count_percent(Data)

# Exploratory Data Analysis EDA + Feature engineering

In [7]:
#Variable Comapny :
print(Data["Company"].isnull().sum())
print(Data["Company"].unique())
fig, ax  = plt.subplots(figsize=(10,5))
ax=sns.countplot(x='Company', data=Data, palette='mako_r', order = Data['Company'].value_counts().index)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.bar_label(ax.containers[0])
plt.title('Number of laptops by brands')
plt.show()

In [8]:
Data.head()

In [9]:
fig, ax  = plt.subplots(figsize=(10,5))
ax=sns.countplot(x='TypeName', data=Data, palette='mako_r', order = Data['TypeName'].value_counts().index)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.bar_label(ax.containers[0])
plt.title('Laptop types')
plt.show()

In [10]:
fig, ax  = plt.subplots(figsize=(10,5))
ax=sns.countplot(x='Inches', data=Data, palette='viridis_r')
ax.set_xticklabels(ax.get_xticklabels(), rotation=80);
ax.bar_label(ax.containers[0])
plt.title('Laptop screen size (inches)')
plt.show()

In [11]:
Liste_gpu = Data['Gpu'].value_counts()[:15].index.tolist()
plt.figure(figsize=(8,6))
ax=sns.countplot(x='Gpu', data=Data[Data['Gpu'].isin(Liste_gpu)], order = Liste_gpu, palette='viridis_r')
plt.xticks(rotation=80);
ax.bar_label(ax.containers[0])
plt.title('TOP 15 common GPUs')
plt.xlabel('')
plt.show()

In [12]:
Cpu_liste = Data['Cpu'].value_counts()[:15].index.tolist()
plt.figure(figsize=(8,6))
ax=sns.countplot(x='Cpu', data=Data[Data['Cpu'].isin(Cpu_liste)], order = Cpu_liste, palette='viridis')
plt.xticks(rotation=80);
ax.bar_label(ax.containers[0])
plt.title('TOP 15 common CPUs')
plt.xlabel('')
plt.show()


In [13]:
company_list = Data['Company'].value_counts().index[:8].tolist()
company_list

In [14]:

plt.figure(figsize=(9,5))
ax=sns.barplot(x='Company', y='Price', data=Data[Data['Company'].isin(company_list)],
                order=company_list, 
                palette='Spectral', 
                ci=False,
                edgecolor="black")
plt.xticks(rotation=80);
ax.bar_label(ax.containers[0])
plt.title('Average price of laptops by company')
plt.show()

In [15]:
#variable 
Data["Weight"]=Data["Weight"].str.rstrip('kg').astype("float")
Data["Weight"].head(20)

In [16]:
#RAM
Data['Ram'].value_counts().plot(kind='bar')
Data["Ram"]=Data["Ram"].str.rstrip('GB').astype("int")
Data["Ram"].head(20)

**Variable résolution**

In [17]:
Data['ScreenResolution'].head(40)

In [18]:
#Creation d'une nouvelle variable Touche screen a partir du variable screenResolution
Data['Touchscreen']=Data['ScreenResolution'].apply(lambda x:1 if'Touchscreen' in x else 0)

In [19]:
Data['Touchscreen'].value_counts().plot(kind='bar')
plt.title('Touchscreen count')

In [20]:
Data['ScreenResolution'].head()

In [21]:
Data['Inches']

In [22]:
#Variable Pixel par pouce
new = Data['ScreenResolution'].str.split('x',n=1,expand=True)
Data['X_res'] = new[0]
Data['Y_res'] = new[1]
Data['X_res'] = Data['X_res'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x:x[0])
Data['X_res'] = Data['X_res'].astype('int')
Data['Y_res'] = Data['Y_res'].astype('int')
Data['ppi'] = (((Data['X_res']**2) + (Data['Y_res']**2))**0.5/Data['Inches']).astype('float')
Data.head()
Data.drop(columns=['X_res','Y_res'],inplace=True)

In [23]:
Data['ppi']

In [24]:
Data['Screentype']=Data['ScreenResolution'].replace(r'(\d+x\d+)','',regex=True)
Data['Screentype'].head(50)

In [25]:
#Data['Screentype'] = Data['Screentype'].replace(r'(Full HD|Quad HD|Quad HD|\+|/|4K Ultra HD)','',regex=True)
#Data['Screentype'].head(50)

In [26]:
#Memory
Data['Memory'].value_counts()

In [27]:
Data['Memory'].head(40)

# Variable Memory

In [28]:
Data['Memory'] = Data['Memory'].astype(str).replace('\.0', '', regex=True)
Data["Memory"] = Data["Memory"].str.replace('GB', '')
Data["Memory"] = Data["Memory"].str.replace('TB', '000')
Data['Memory'].value_counts()
new = Data["Memory"].str.split("+", n = 1, expand = True)
Data["first"]= new[0]
Data["first"]=Data["first"].str.strip()
Data["second"]= new[1]
Data["Layer1HDD"] = Data["first"].apply(lambda x: 1 if "HDD" in x else 0)
Data["Layer1SSD"] = Data["first"].apply(lambda x: 1 if "SSD" in x else 0)
Data["Layer1Hybrid"] = Data["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
Data["Layer1Flash_Storage"] = Data["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)

Data["second"].fillna("0", inplace = True)
Data["Layer2HDD"] = Data["second"].apply(lambda x: 1 if "HDD" in x else 0)
Data["Layer2SSD"] = Data["second"].apply(lambda x: 1 if "SSD" in x else 0)
Data["Layer2Hybrid"] = Data["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
Data["Layer2Flash_Storage"] = Data["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
Data['second'] = Data['second'].str.replace(r'\D', '')
Data['first'] = Data['first'].str.replace(r'\D', '')
Data['first'].head(50)
Data["first"] = Data["first"].astype(int)
Data["second"] = Data["second"].astype(int)
Data["HDD"]=(Data["first"]*Data["Layer1HDD"]+Data["second"]*Data["Layer2HDD"])
Data["SSD"]=(Data["first"]*Data["Layer1SSD"]+Data["second"]*Data["Layer2SSD"])
Data["Hybrid"]=(Data["first"]*Data["Layer1Hybrid"]+Data["second"]*Data["Layer2Hybrid"])
Data["Flash_Storage"]=(Data["first"]*Data["Layer1Flash_Storage"]+Data["second"]*Data["Layer2Flash_Storage"])
Data.drop(columns=['Memory','first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
       'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
       'Layer2Flash_Storage'],inplace=True)


In [29]:
Data.head(30)

In [30]:
Data.head()

# Variable Gpu

In [31]:
Data['Gpu'].value_counts()

In [32]:
Data['Gpu'].value_counts()
Data['Gpu_brand'] =Data.Gpu.str.extract(r'(Nvidia|AMD|Intel|ARM)')
Data['Gpu_brand'].value_counts()
Data[Data['Gpu_brand']=='ARM']

In [33]:
#varibale ops
Data['OpSys'].value_counts()

In [34]:
def catageorie_os(os):
    if os == 'Windows 10' or os == 'Windows 7' or os == 'Windows 10 S':
        return 'Windows'
    elif os == 'macOS' or os == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others/No OS/Linux'


In [35]:
Data['OpSys']=Data['OpSys'].apply(catageorie_os)

Data.head(20)

In [36]:

Data['Cpu'].head(10)

In [37]:
#Cpu
Data['Cpu'].value_counts()
Data['Cpu_Name'] = Data['Cpu'].apply(lambda x:" ".join(x.split()[0:3]))
def fetch_processor(text):
    if text == 'Intel Core i7' or text == 'Intel Core i5' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'
Data['Cpu_Name']=Data['Cpu_Name'].apply(fetch_processor)
Data['cpu_freq(GHz)'] = Data['Cpu'].str.extract(r'(\d+\.\d+GHz|\d+GHz)')
Data['cpu_freq(GHz)'] = Data['cpu_freq(GHz)'].str.replace('GHz','').astype('float64')
Data.drop(columns=['Cpu'],inplace=True)
#Data.rename(columns={'Cpu_freq': 'cpu_freq(GHz)'}, inplace=True)


In [38]:
Data['cpu_freq(GHz)']

In [39]:
sns.barplot(x=Data['Cpu_Name'],y=Data['Price'])
plt.xticks(rotation='vertical')
plt.show()

In [40]:
Data.head()

# Categorical features encoding

In [41]:
copy_Data=Data.copy()
Save_data=Data.copy()

In [42]:
Data.info()

In [43]:
from sklearn.preprocessing import LabelEncoder
catCols =  ['Company','TypeName','Gpu','OpSys','Screentype','Gpu_brand','Cpu_Name','ScreenResolution']
#One hot encoding
print('Dataframe encoded by OHE dimension : ', pd.get_dummies(Save_data, columns=catCols, drop_first=True).shape)
#Label Encoding will be performed to keep the number of features lower.
#Label encoding
en = LabelEncoder()
for cols in catCols:
    Data[cols] = en.fit_transform(Data[cols])
print('Dataframe encoded by Label encoding dimension : ', Save_data.shape)

In [44]:
import seaborn as sns
import numpy as np
matrix = np.triu(Data.corr())
plt.figure(figsize=(13, 10))
sns.heatmap(Data.corr(), annot = True, fmt=".2f", mask = matrix,
            vmin = -1, vmax = 1, linewidths = 0.1, linecolor = 'white', cbar = False)
plt.show()

In [45]:
Save_data['Screentype'].unique()

In [46]:
Save_data['cpu_freq(GHz)'].unique() 

In [47]:
Save_data['ScreenResolution']

In [48]:
Data.drop(columns=['Gpu','ScreenResolution','Inches','Screentype','Hybrid','Flash_Storage'],inplace=True)
Save_data.drop(columns=['Gpu','ScreenResolution','Inches','Screentype','Hybrid','Flash_Storage'],inplace=True)
copy_Data.drop(columns=['Gpu','ScreenResolution'],inplace=True)

In [49]:
Data.head(2)


In [50]:
Data.corr()['Price']

In [51]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(Data.corr()[['Price']].sort_values(by='Price', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Price', fontdict={'fontsize':18}, pad=16);

In [52]:
def num_plot(df, col, title, symb):
    fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8,5),gridspec_kw={"height_ratios": (.2, .8)})
    ax[0].set_title(title,fontsize=18)
    sns.boxplot(x=col, data=df, ax=ax[0])
    ax[0].set(yticks=[])
    sns.histplot(x=col, data=df, ax=ax[1])
    ax[1].set_xlabel(col, fontsize=16)
    plt.axvline(df[col].mean(), color='darkgreen', linewidth=2.2, label='mean=' + str(np.round(df[col].mean(),1)) + symb)
    plt.axvline(df[col].median(), color='red', linewidth=2.2, label='median='+ str(np.round(df[col].median(),1)) + symb)
    plt.axvline(df[col].mode()[0], color='purple', linewidth=2.2, label='mode='+ str(df[col].mode()[0]) + symb)
    plt.legend(bbox_to_anchor=(1, 1.03), ncol=1, fontsize=17, fancybox=True, shadow=True, frameon=True)
    plt.tight_layout()
    plt.show()

In [53]:
num_plot(Data, 'Price', 'Price Distribution','$')

In [54]:
Data.drop(columns=Data.columns[0],inplace=True)
Save_data.drop(columns=Save_data.columns[0],inplace=True)
Data.head()

# Ml modeling

In [55]:
def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """Scatter plot of the predicted vs true targets."""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],
            '--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('True_Price')
    ax.set_ylabel('Predicted_Price')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    title = title + 'n Evaluation in {:.2f} seconds'.format(elapsed_time)
    ax.set_title(title)

In [56]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split 
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [57]:
X = Data.drop(columns=['Price'])
y = np.log(Data['Price'])

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [60]:
Data.shape

In [61]:
Data.info()

In [62]:
#LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred))
print("R2 using Linear Regression: {:.2f} %".format(np.round(r2_score(y_test, y_pred),4)*100))
print("RMSE using LR: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(model)
print(score)
print("LinearRegression score with crossvalidation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [63]:
#Ridge Regression
ridge = Ridge(alpha=10)
ridge.fit(X_train,y_train)
y_pred_r = ridge.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_r))
print("R2 using Ridge Regression: {:.2f} %".format(np.round(r2_score(y_test, y_pred_r),4)*100))
print("RMSE using RR: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(ridge)
print(score)
print("Ridge Regression score with crossvalidation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [64]:
#Lasso Regression
lasso = Lasso(alpha=0.001)
lasso.fit(X_train,y_train)
y_pred_ls = lasso.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_ls))
print("R2 using Linear Regression: {:.2f} %".format(np.round(r2_score(y_test, y_pred_ls),4)*100))
print("RMSE using LR: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(lasso)
print("Lasso Regression score with crossvalidation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [65]:
#KNN 
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_knn))
print("R2 using Knn: {:.2f} %".format(np.round(r2_score(y_test, y_pred_knn),4)*100))
print("RMSE using Knn: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(model)
print("Knn score with crossvalidation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [66]:
#svr 
svr = SVR(kernel='rbf',C=10000,epsilon=0.1)
svr.fit(X_train,y_train)
y_pred_svr = svr.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_svr))
print("R2 using Knn: {:.2f} %".format(np.round(r2_score(y_test, y_pred_svr),4)*100))
print("RMSE using Knn: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(svr)
print("SVM score with cross validation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [67]:
#Random Forest 
rf = RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_samples=0.5,
                              max_features=0.75,
                              max_depth=15)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_rf))
print("R2 using Random Forest : {:.2f} %".format(np.round(r2_score(y_test, y_pred_rf),4)*100))
print("RMSE using Random Forest : {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(rf)
print("Random Forest score with cross validation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [68]:
#XgBoost
model = XGBRegressor(n_estimators=45,max_depth=5,learning_rate=0.5)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred))
print("R2 using XgBoost: {:.2f} %".format(np.round(r2_score(y_test, y_pred),4)*100))
print("RMSE using XgBoost: {} ".format(np.round(np.sqrt(mse)),4))

In [69]:
#Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=500)
gbr.fit(X_train,y_train)
y_pred_gbr = gbr.predict(X_test)
mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_gbr))
print("R2 using XgBoost: {:.2f} %".format(np.round(r2_score(y_test, y_pred_gbr),4)*100))
print("RMSE using XgBoost: {} ".format(np.round(np.sqrt(mse)),4))
score = rmsle_cv(gbr)
print("Gradient Boosting Regressor with cross validation: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [70]:
feature_name_list=Data.drop('Price', axis = 1).columns
feature_name_list
rf.feature_names = feature_name_list
plt.barh(rf.feature_names,rf.feature_importances_)
plt.xticks(rotation=90);
plt.title('Feature Importance by Random Forest')
plt.xlabel('Feature Importance (%)')
plt.show()

In [71]:
from sklearn.model_selection import train_test_split
# to handle datasets
import pandas as pd
import numpy as np
# for plotting
import matplotlib.pyplot as plt
# to divide train and test set
from sklearn.model_selection import train_test_split
# feature scaling
from sklearn.preprocessing import MinMaxScaler
#to buid models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
# models for Stacking
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
# to evaluate the model
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error,r2_score
import math
#to find training time of the model
import time
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
import warnings
#X = Save_data.drop(columns=['Price'])
#y = np.log(Save_data['Price'])
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)


In [72]:
X = Data.drop(columns=['Price'])
y = np.log(Data['Price'])
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [73]:
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100,random_state=3,max_samples=0.5,max_features=0.75,max_depth=15)),
    ('gbdt',GradientBoostingRegressor(n_estimators=500)),
    #('lasso', Lasso(alpha=0.001)),
    ('knn',KNeighborsRegressor(n_neighbors=3)),
    ('xgb', XGBRegressor())]
     
stacking_regressor= StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=100))

fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)
errors_list=[]
for ax, (name, est) in zip(axs, estimators + [('Stacking Regressor',stacking_regressor)]):
    start_time = time.time()
    model = est.fit(X_train, y_train)
                     
    elapsed_time = time.time() - start_time
    
    pred = model.predict(X_test)
    errors = y_test - model.predict(X_test)
    errors_list.append(errors)
    test_r2= r2_score(np.exp(y_test), np.exp(pred))
    
    test_rmsle=math.sqrt(mean_squared_log_error(y_test,pred))
    plot_regression_results(ax,y_test,pred,name,(r'$R^2={:.3f}$' + '\n' + 
                            r'$RMSLE={:.3f}$').format(test_r2,test_rmsle),elapsed_time)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [74]:
#Best model - Ridge regression
best_model = gbr
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print('R2:', r2_score(y_test, y_pred))
print('mse:', mean_squared_error(y_test, y_pred))
print('mae:', mean_absolute_error(y_test, y_pred))
plt.figure(figsize = (10, 6))
plt.plot(np.arange(len(y_test)), y_test, label = 'y_test')
plt.plot(np.arange(len(y_test)), y_pred, label = 'y_pred')
plt.xlabel('')
plt.ylabel('Price')
plt.legend()
plt.title('Comparison y_test and y_train')
plt.show()

In [75]:
#Exporting the Pipeline and Dataset
import pickle
pickle.dump(best_model,open('best_model.pkl','wb'))
pickle.dump(Save_data,open('Save_data.pkl','wb'))

**L'utilisation de onehot coding method avec pipline pour l'utiliser apres avec le streamlit **

In [76]:
Data.head(1)

In [77]:
#!pip install -q streamlit

In [78]:
Save_data

In [79]:
from sklearn.model_selection import train_test_split
X = Save_data.drop(columns=['Price'])
y = np.log(Save_data['Price'])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [80]:
X_train

In [81]:
tr1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')
tr2 = KNeighborsRegressor(n_neighbors=5)
pipe = Pipeline([
    ('tr1',tr1),
    ('tr2',tr2)
])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

# Linear regression

In [82]:
tr1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')

tr2 = LinearRegression()
pipe = Pipeline([
    ('tr1',tr1),
    ('tr2',tr2)
])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

In [83]:
Save_data.head(1)

In [84]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')

step2 = Ridge(alpha=10)

pipe1 = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe1.fit(X_train,y_train)

y_pred = pipe1.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))


# Ridge Regression

In [85]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')

step2 = Ridge(alpha=10)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

# Lasso Regression

In [86]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')

step2 = Lasso(alpha=0.001)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

In [87]:
Save_data.head(1)

In [88]:
X_train

In [89]:
from sklearn.ensemble import VotingRegressor,StackingRegressor

step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,3,9,10])
],remainder='passthrough')


estimators = [
    ('rf', RandomForestRegressor(n_estimators=350,random_state=3,max_samples=0.5,max_features=0.75,max_depth=15)),
    ('gbdt',GradientBoostingRegressor(n_estimators=100,max_features=0.5)),
    ('lasso', Lasso(alpha=0.001))
   #('xgb', XGBRegressor())
]

step2 = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=100))

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print("R2: {:.2f} %".format(np.round(r2_score(y_test,y_pred),4)*100))
print('MAE',mean_absolute_error(y_test,y_pred))

In [90]:
step2

In [91]:
#Exporting the Pipeline and Dataset
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))
pickle.dump(Save_data,open('Save_data.pkl','wb'))

In [ ]:
Save_data.info()

In [ ]:
Save_data.head(2)

In [ ]:
from sklearn.externals import joblib
m1=joblib.load('pipe.pkl')

In [ ]:
import xgboost as xgb
print(xgb.__version__)
print(shap.__version__)